# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
from collections import deque
import torch

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size

print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.7999999821186066


When finished, you can close the environment.

In [20]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
def ddpg(n_episodes=1000, max_t=300, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0] 
        agent.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state)
#             action = np.clip(action, -1, 1)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0] 
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores



In [6]:
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)

In [7]:
scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

[0.16665371 0.19530685 0.02198273 0.0538726 ]
[0.30486745 0.31424555 0.15438218 0.11311731]
[0.37644523 0.3895554  0.24915245 0.13357185]
[0.40217254 0.4109785  0.358039   0.31817606]
[0.52774435 0.45925224 0.39475307 0.32966423]
[0.45172647 0.39676148 0.4298279  0.34967136]
[0.45591897 0.5162742  0.47172365 0.41504776]
[0.43074098 0.44416073 0.46718857 0.38581827]
[0.46416813 0.57798624 0.532938   0.36989924]
[0.569108   0.65150565 0.6007438  0.50146914]
[0.6319295  0.71270865 0.5819926  0.6284265 ]
[0.72504216 0.639345   0.64612883 0.68341655]
[0.7039245  0.6504286  0.64759785 0.7723387 ]
[0.69346493 0.7200374  0.6217027  0.83926713]
[0.76435554 0.7058134  0.6424859  0.90371436]
[0.7894862  0.69876796 0.5916773  0.8395095 ]
[0.8058268  0.62871766 0.68599176 0.7736446 ]
[0.8615492  0.59756744 0.77680117 0.80531925]
[0.8276342  0.612359   0.79297006 0.8086136 ]
[0.75967187 0.56302005 0.7788437  0.8810132 ]
[0.7641421  0.4946333  0.82869893 0.90067685]
[0.8254665 0.4596263 0.8553147 0.7

/Users/ramtinr/Desktop/Udacity-DRLND/code/drlnd-continuous-control/ddpg_agent.py:104: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


[0.61630446 0.5902078  0.492857   0.84355664]
[0.6516111  0.6496798  0.44938144 0.8699194 ]
[0.63251346 0.7427415  0.5441019  0.85942847]
[0.6389195 0.6500644 0.4881884 0.7410279]
[0.7505542  0.7025951  0.58154064 0.6264323 ]
[0.7180232  0.74342954 0.6667576  0.7196738 ]
[0.6716007  0.7099083  0.5648489  0.66790533]
[0.6684399 0.6047256 0.4599877 0.6701751]
[0.58590037 0.51813763 0.5131505  0.6531117 ]
[0.5354304  0.5442985  0.52644104 0.72975034]
[0.60953534 0.49667352 0.4743422  0.7350975 ]
[0.5969385  0.4198815  0.38682985 0.7145511 ]
[0.6334563  0.4046268  0.39926884 0.72276676]
[0.6267753  0.5019668  0.4567672  0.77270633]
[0.6175147  0.4898243  0.57498777 0.8406646 ]
[0.65615267 0.5796157  0.5073677  0.76674575]
[0.6816679  0.6429401  0.47493848 0.84381604]
[0.6641786  0.6326443  0.45647296 0.9055634 ]
[0.71267664 0.5662782  0.4549027  0.79146063]
[0.71938026 0.67849517 0.48267785 0.8298607 ]
[0.8035549 0.7958566 0.4305629 0.7372793]
[0.74347246 0.7297574  0.35647535 0.6677895 ]


[-0.11452472  0.89885414  1.          1.        ]
[-0.1035281  1.         1.         1.       ]
[-0.15498671  1.          1.          1.        ]
[-0.25000432  1.          1.          1.        ]
[0.02022319 1.         1.         1.        ]
[0.08603971 1.         1.         1.        ]
[0.11436986 1.         1.         1.        ]
[-0.02603649  1.          1.          1.        ]
[-0.00830263  1.          1.          1.        ]
[0.01811119 1.         1.         1.        ]
[0.08780916 1.         1.         1.        ]
[0.1222702 1.        1.        1.       ]
[0.11669923 1.         1.         1.        ]
[0.07241853 1.         1.         1.        ]
[-0.0287142  1.         1.         1.       ]
[-0.15126622  1.          1.          1.        ]
[-0.13919316  1.          1.          1.        ]
[-0.07059228  1.          1.          1.        ]
[-0.13896783  1.          1.          1.        ]
[-0.15783715  1.          1.          1.        ]
[-0.2327213  1.         1.         1.       

[-0.3796448   0.12957868  1.          0.87281084]
[-0.35179886  0.15261686  1.          1.        ]
[-0.43675086  0.5193695   1.          1.        ]
[-0.53789383  0.8417748   1.          1.        ]
[-0.5294773  1.         1.         1.       ]
[-0.4434203  1.         1.         1.       ]
[-0.45148566  1.          1.          1.        ]
[-0.48397532  1.          1.          1.        ]
[-0.3631356  1.         1.         1.       ]
[-0.38325658  1.          1.          1.        ]
[-0.3382834  1.         1.         1.       ]
[-0.24432407  1.          1.          1.        ]
[-0.2257828  1.         1.         1.       ]
[-0.25616616  1.          1.          1.        ]
[-0.30784193  1.          1.          1.        ]
[-0.22601713  1.          1.          1.        ]
[-0.15374625  1.          1.          1.        ]
[-0.17283714  1.          1.          1.        ]
[-0.16145864  1.          1.          1.        ]
[-0.09805481  1.          1.          1.        ]
[-0.09189097  1.    

...

[ 0.90882885 -0.2517829   1.         -0.3227276 ]
[ 0.21103074 -0.3466713   1.         -0.02271258]
[-0.23663394 -0.43342364  1.          1.        ]
[-0.32050714 -0.48766243  1.          1.        ]
[-0.26905015 -0.40679288  1.          1.        ]
[-0.24978086  0.17971684  1.          1.        ]
[-0.25617868  0.4655313   1.          1.        ]
[-0.34292975  0.5290182   1.          1.        ]
[-0.36278567  0.4763601   1.          1.        ]
[-0.3174761   0.36735576  1.          1.        ]
[-0.27702615  0.39182726  1.          1.        ]
[-0.32087132  0.3600185   1.          1.        ]
[-0.2959986   0.33168644  1.          1.        ]
[-0.2692607   0.33690515  1.          1.        ]
[-0.31122917  0.3718881   1.          1.        ]
[-0.27326432  0.31899947  1.          1.        ]
[-0.29016826  0.33518952  1.          1.        ]
[-0.21654914  0.3729218   1.          1.        ]
[-0.15371738  0.6093394   1.          1.        ]
[-0.26232198  0.88259596  1.          1.        ]


[ 1.         -0.5001228   1.         -0.42669913]
[ 1.         -0.5232058   1.         -0.40005836]
[ 1.         -0.567291    1.         -0.31991243]
[ 1.         -0.44883177  1.         -0.40036467]
[ 1.         -0.3672693   1.         -0.31336075]
[-0.14185913 -0.29940665  1.          1.        ]
[-0.28889793 -0.3551932   1.          1.        ]
[-0.26998743 -0.44646242  1.          1.        ]
[ 0.02009619 -0.4912538   1.          1.        ]
[ 0.31442514 -0.519335    1.          1.        ]
[ 0.6515566  -0.45762965  1.          1.        ]
[ 0.86623496 -0.35272977  1.          1.        ]
[ 0.76426625 -0.26008108  1.          1.        ]
[ 0.70776093 -0.2124035   1.          1.        ]
[ 0.5553549  -0.26412562  1.          1.        ]
[ 0.5161898  -0.33039352  1.          1.        ]
[ 0.4722762  -0.25995058  1.          1.        ]
[ 0.32072607 -0.22397916  1.          1.        ]
[ 0.11044626 -0.20406853  1.          1.        ]
[-0.19502552 -0.07128674  1.          1.        ]


[-0.51097673  0.4686091   1.          0.46475938]
[-0.56571066  0.5731398   1.         -0.04814942]
[-0.51823187  0.6051921   1.         -0.18809575]
[-0.3849073   0.60130847  1.         -0.2505007 ]
[-0.34857413  0.6091677   1.         -0.25739333]
[-0.4146761   0.4810649   1.         -0.23556565]
[-0.3532113  0.478383   1.        -0.2761595]
[-0.3968686   0.45837653  1.         -0.3606538 ]
[-0.4069999   0.4750832   1.         -0.40830162]
[-0.31547546  0.6110106   1.         -0.2416854 ]
[-0.37451363  0.60738826  1.          0.0456507 ]
[-0.44515648  0.4626948   1.          0.27228397]
[-0.40437528  0.27495477  1.          0.50038666]
[-0.43646145  0.13077882  1.          0.3383539 ]
[-0.4041494  -0.10816567  1.          0.3621315 ]
[-0.441619   -0.16206898  1.          0.6136887 ]
[-0.36956027 -0.27457651  1.          0.6961769 ]
[-0.40445572 -0.01843649  1.          1.        ]
[-0.4888131  0.646151   1.         1.       ]
[-0.40478802  0.1488155   1.          0.639482  ]
[-0.3043

[-0.38714275 -0.1588767   1.          1.        ]
[-0.4423824  -0.20704228  1.          1.        ]
[-0.50268936 -0.17991947  1.          1.        ]
[-0.561449   -0.16054432  1.          1.        ]
[-0.42818832 -0.25902018  1.          1.        ]
[-0.43015158 -0.2949655   1.          1.        ]
[-0.43519253 -0.39157248  1.          1.        ]
[-0.43315616 -0.30305582  1.          0.40625912]
[-0.34123084 -0.38408014  1.         -0.06495956]
[-0.42492235 -0.32620248  1.          0.3215776 ]
[-0.42518055 -0.26828057  1.          1.        ]
[-0.3221239  -0.35328367  1.          1.        ]
[-0.3728273 -0.4196476  1.         1.       ]
[-0.27444726 -0.3412036   1.          1.        ]
[-0.32351673 -0.3874739   1.          1.        ]
[-0.3593493  -0.13012882  1.          1.        ]
[-0.29337326  1.          1.          1.        ]
[-0.31126913  1.          1.          1.        ]
[-0.3526876  1.         1.         1.       ]
[-0.37238955  1.          1.          1.        ]
[-0.2951

KeyboardInterrupt: 